In [34]:
from os import listdir
from os.path import isfile, join
import numpy
import cv2
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import layers
from keras.layers import Input, Dense, Activation,BatchNormalization, Flatten, Conv2D, MaxPooling2D ,Dropout
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow import keras
import keras
from keras.models import Sequential
from sklearn.model_selection import KFold
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D,LSTM
from sklearn.metrics import plot_confusion_matrix,roc_curve, roc_auc_score
import tensorflow as tf
import numpy as np
from matplotlib.pyplot import imshow
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report 
import re
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

In [35]:
# #images = np.load('/content/drive/MyDrive/images.npy')
# def extract_number(string):
#     r = re.compile(r'(\d+)')
#     return int(r.findall(string)[0])


# dim = (300, 300)
# mypath= r'C:\Users\phmeay\Desktop\Work files\DTF-PDC\OPDC\OPDC\image files'
# onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
# sortedFiles = sorted(onlyfiles ,key=lambda x: extract_number(x) )
# images = np.empty((len(sortedFiles),300,300))
# for n in range(0, len(sortedFiles)):
#   img = cv2.imread( join(mypath,sortedFiles[n]),cv2.IMREAD_GRAYSCALE  )
#   images[n] = cv2.resize(img, dim)
# label_motion = np.ones((len(images[:,1,1]))-952)
# label_therapy = np.zeros((len(images[:,1,1]))-663)
# y = np.hstack((label_motion,label_therapy))

# print(images.shape)
# print(y.shape)

In [74]:
#images = np.load('/content/drive/MyDrive/images.npy')
def extract_number(string):
    r = re.compile(r'(\d+)')
    return int(r.findall(string)[0])


dim = (300, 300)
mypath= r'C:\Users\phmeay\Desktop\Work files\DTF-PDC\OPDC\OPDC\images file'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
sortedFiles = sorted(onlyfiles ,key=lambda x: extract_number(x) )
# images = numpy.empty(len(sortedFiles), dtype=object)
images =[]
for n in range(0, len(sortedFiles)):
  img  = cv2.imread( join(mypath,sortedFiles[n]),cv2.IMREAD_GRAYSCALE  )
  images.append(img)
  # images[n] = cv2.resize(img, dim)
images =np.array(images)
label_motion = np.ones((len(images[:,1,1]))-952)
label_therapy = np.zeros((len(images[:,1,1]))-663)
y = np.hstack((label_motion,label_therapy))

print(images.shape)
print(y.shape)

(1615, 656, 875)
(1615,)


In [75]:
X= images.reshape((images.shape[0], 656*875))

In [76]:
num_classes =2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=8)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1211, 574000)
(404, 574000)
(1211,)
(404,)


In [77]:
# X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.30, random_state=1) # 0.25 x 0.8 = 0.2
# print(X_train.shape)
# print(X_val.shape)
# print(y_train.shape)
# print(y_val.shape)

In [78]:
# convert from integers to floats
train_norm = X_train.astype('float32')
test_norm = X_test.astype('float32')
# normalize to range 0-1
X_train = train_norm / 255.0
X_test = test_norm / 255.0

In [79]:
pca = PCA(0.9)

In [80]:
pca.fit(X_train)

PCA(n_components=0.9)

In [81]:
pca.n_components_

41

In [82]:
train_img_pca = pca.transform(X_train)
test_img_pca = pca.transform(X_test)
# val_img_pca = pca.transform(X_val)
print(train_img_pca.shape)
print(test_img_pca.shape)
# print(val_img_pca.shape)

(1211, 41)
(404, 41)
(388, 41)


In [83]:
batch_size = 128
num_classes = 2
epochs = 30

In [85]:
############### using K-Fold #########################


# inputs = np.concatenate((train_img_pca, val_img_pca), axis=0)
# targets = np.concatenate((y_train, y_val), axis=0)
inputs =train_img_pca
targets = y_train

# Model configuration

batch_size = 50
no_epochs = 50

verbosity = 1
num_folds = 5
# Define the K-fold Cross Validator
kfold_traning = KFold(n_splits=num_folds, shuffle=True)


acc_per_fold = []
loss_per_fold = []
# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold_traning.split(inputs, targets):
    
  # Define the model architecture
    model = Sequential()
    model.add(Dense(96, activation='relu', input_shape=(41,)))
    model.add(Dense(64, activation='relu'))
#     model.add(Dense(512, activation='relu'))
#     model.add(Dense(256, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))


  # Compile the model
    model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])


    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
  
    # Fit data to model
    history = model.fit(inputs[train], targets[train],
                batch_size=batch_size,
                epochs=no_epochs,
                verbose=0)
  
    # Generate generalization metrics
    
    scores_test = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores_test[0]}; {model.metrics_names[1]} of {scores_test[1]*100}%')
    acc_per_fold.append(scores_test[1] * 100)
    loss_per_fold.append(scores_test[0])
    
  
    # Increase fold number
    fold_no = fold_no + 1
final_score = model.evaluate(test_img_pca, y_test, verbose=0)
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss_val: {loss_per_fold[i]} - Accuracy_val: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

print(final_score)
####################################################################################################################

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 0.15966983139514923; accuracy of 92.18106865882874%
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 0.110819973051548; accuracy of 95.04132270812988%
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 0.3095683753490448; accuracy of 93.388432264328%
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 0.10456747561693192; accuracy of 96.28099203109741%
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 0.20280872285366058; accuracy of 94.2148745059967%
------------------------------------------------------------------------
Score per fold
-------------------------------------------